#TP3

Paulo Freitas - A100053


Pedro Santos - A100110

### Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $$N>0$$

    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'
    OUTPUT r, s, t


Exercício 1

Este exercício é dirigido às provas de segurança do algoritmo acima.

  1. Construa um FOTS   $$\Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$$ usando este modelo nos inteiros.
  2. Considere como propriedade de segurança:
                   safety = (r > 0) and (r < N) and (r = a*``s + b*t`)
  Prove usando k-indução que esta propriedade se verifica em qualquer traço do FOTS


  3. Prove usando “Model-Checking” com interpelantes e invariantes prove também que esta propriedade é um invariante em qualquer traço de $\,\Sigma\,$.

  #### Nota: De momento o uso de interpolantes não é possível em `z3` e requer um dos solvers  `msat` ou `yices`. A experiência mostra que  o `pysmt`  com  Python 3.11 ou 3.12  não instala qualquer destes “solvers”;  para isso exige-se uma instalação com o Python 3.10. |

Exercício 2

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

  1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função gcd é:   $$\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$$
  2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
  3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”.
  4. Prove a correção  do programa LPA em ambas as codificações.


# Import das Bibliotecas

Para este exercício serão usadas as bibliotecas do pysmt e os sover z3 e msat.

In [1]:
%%capture
!yes | pip install pysmt
!apt-get install libgmp3-dev
!yes | pysmt-install --z3 --msat

file = '/usr/local/lib/python3.10/dist-packages/pysmt/smtlib/parser/__init__.py'
with open(file, 'r') as f:
    code = f.read()
    new_code = code.replace('USE_CYTHON = True', 'USE_CYTHON = False')

with open(file, 'w') as f:
  f.write(new_code)

In [17]:
from pysmt.shortcuts import *
from pysmt.typing import INT
from pysmt.logics import QF_NIA
import itertools

## EXERCÍCIO 1


### Exercício 1.1

Construa um FOTS   $$\Sigma\,\equiv\,\langle\mathsf{X}, \mathsf{I},\mathsf{T}\rangle\,$$ usando este modelo nos inteiros.

#### Variáveis

Este FOTS é uma adaptação do FOTS realizado no trabalho passado, sendo que a maior alteração é a mudança de tipos onde BVtype passa a Int.


Tomemos de exemplo os inteiros de input n = 40, a = 20 e b = 5.

In [3]:
n=40

a=20
b=15

#mdc(a,b)

#### Declaração
A função *declare* declara as variáveis existentes em cada estado.

In [4]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),INT)

    state['r'] = Symbol('r'+str(i),INT)
    state['r_'] = Symbol('r_'+str(i),INT)
    state['s'] = Symbol('s'+str(i),INT)
    state['s_'] = Symbol('s_'+str(i),INT)
    state['t'] = Symbol('t'+str(i),INT)
    state['t_'] = Symbol('t_'+str(i),INT)
    state['q'] = Symbol('q' +str(i),INT)

    return state

#### Inicialização
A função *init* inicializa as variáveis de um dado estado. Segue a inicialização por base das informações presentes no enunciado.

In [5]:
def init(state):
    E = Equals(state['pc'],Int(0))

    R = Equals(state['r'],Int(a))
    R_ = Equals(state['r_'],Int(b))
    S = Equals(state['s'], Int(1))
    S_ = Equals(state['s_'],Int(0))
    T = Equals(state['t'],Int(0))
    T_ = Equals(state['t_'],Int(1))
    Q = Equals(state['q'],Int(0))

    return And(E,R,R_,S,S_,T,T_)

#### Transição

Seja o ciclo presente no enuniado, com a alteração em que cada estado do ciclo tem um número associado, ou seja:


    0:while r' != 0
    1:  q = r div r'
        r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'
    2: Outup

Na função *trans* são dadas as possiveis transições de estado do ciclo. O programa só pode entrar no ciclo se estiver no estado 0 (t01), o programa só pode sair do ciclo se verificar a propriedade 0 (t02), o programa so pode voltar ao estado 0 após executar o código 1 (t10) e o programa estando no estado 2 apenas pode continuar no estado 2.

In [6]:
def trans(curr,prox):

    t01 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr['r_'],Int(0))), Equals(prox['pc'],Int(1)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t02 = And(Equals(curr['pc'],Int(0)), Equals(curr['r_'],Int(0)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t10 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(0)),
              Equals(curr['q'], Div(curr['r'],curr['r_'])),
              Equals(prox['r'],curr['r_']), Equals(prox['r_'], Minus(curr['r'],Times(curr['q'],curr['r_']))),
              Equals(prox['s'],curr['s_']), Equals(prox['s_'], Minus(curr['s'],Times(curr['q'],curr['s_']))),
              Equals(prox['t'],curr['t_']), Equals(prox['t_'], Minus(curr['t'],Times(curr['q'],curr['t_']))))

    t22 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    return Or(t01,t02,t10,t22)

#### Euclides

Nesta porção de código, o programa, com o solver, invoca a função *declare* para declarar as variáveis para cada passo do algoritmo. No solver são adicionadas as condições de inicialização do primeiro estado e a função *trans* para todos os estados consecutivos.
Por fim, o programa termina imprimindo as variáveis r,s e t do ultimo estado.

In [36]:
def euclides(declare,init,trans,k):
    with Solver(name="z3") as solver:
      states = [declare(i) for i in range(k)]

      solver.add_assertion(init(states[0]))
      solver.add_assertion(And(trans(states[i],states[i+1]) for i in range(k-1)))

      if solver.solve():
        print("r =",solver.get_value(states[-1]["r"])," s =",solver.get_value( states[-1]["s"])," t =",solver.get_value(states[-1]["t"]))
        print(f'mdc({a},{b}) = {solver.get_value(states[-1]["r"])}')

euclides(declare,init,trans,n)

r = 5  s = 1  t = -1
mdc(20,15) = 5


### Exercício 1.2


  2. Considere como propriedade de segurança:
                        safety = (r > 0) and (r < N) and (r = a*s + b*t)
  Prove usando k-indução que esta propriedade se verifica em qualquer traço do FOTS.

#### Invariante

Começemos por transformar a propriedade safety em um conjunto de invariantes. Esta condição deverá ser verdadeira por k interações.

In [7]:
def inv(state):

  p1 = GT(state['r'],Int(0))
  p2 = LT(state['r'],Int(n))
  p3 = Equals(state['r'], Plus(Times(Int(a),state['s']), Times(Int(b),state['t'])))

  return And(p1,p2,p3)

#### K-Indução

A k-indução procura provar que se para um dado k interações, o problema for satisfazivel a uma dado invariante, então ele permanecerá assim para todo o problema.

In [8]:
def kinduction_always(declare,init,trans,inv,k):
    with Solver(name="z3") as solver:
        states = [declare(i) for i in range(k)]

        solver.add_assertion(init(states[0]))

        solver.add_assertion(And(trans(states[i],states[i+1]) for i in range(k-1)))

        for i in range(k):
            solver.push()
            solver.add_assertion(Not(inv(states[i])))
            if solver.solve():
                print(f"> Contradição! O invariante não se verifica nos k estados iniciais.")
                for st in states:
                    print("x, pc, inv: ", solver.get_value(st['x']), solver.get_value(st['pc']))
                return
            solver.pop()

        state2 = [declare(i+k) for i in range(k+1)]

        for i in range(k):
            solver.add_assertion(inv(state2[i]))
            solver.add_assertion(trans(state2[i],state2[i+1]))

        solver.add_assertion(Not(inv(state2[-1])))

        if solver.solve():
            print(f"> Contradição! O passo indutivo não se verifica.")
            for i,state in enumerate(states):
                print(f"> State {i}: x = {solver.get_value(state['x'])}, pc= {solver.get_value(state['pc'])}.")
            return

        print(f"> A propriedade verifica-se por k-indução (k={k}).")

kinduction_always(declare,init,trans,inv,3)

> A propriedade verifica-se por k-indução (k=3).


### Exercício 1.3

Prove usando “Model-Checking” com interpelantes e invariantes prove também que esta propriedade é um invariante em qualquer traço de $\,\Sigma\,$.

Para a realização deste exercício, será necessário transformar o FOTS em SFOTS.
Isso implica o desenvolvimento de um codigo novo baseado no FOTS já desenvolvido.

In [9]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

def invert(trans):
    return lambda curr, prox: trans(prox, curr)

In [10]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

In [11]:
def init1(s):
    E = Equals(s['pc'],Int(0))

    A = GT(Int(a),Int(0))
    B = GT(Int(b),Int(0))

    R = Equals(s['r'],Int(a))
    R_ = Equals(s['r_'],Int(b))
    S = Equals(s['s'], Int(1))
    S_ = Equals(s['s_'],Int(0))
    T = Equals(s['t'],Int(0))
    T_ = Equals(s['t_'],Int(1))
    Q = Equals(s['q'],Int(0))

    return And(E,A,B,R,R_,S,S_,T,T_)


In [12]:
def trans1(curr,prox):

    t01 = And(Equals(curr['pc'],Int(0)), Not(Equals(curr['r_'],Int(0))), Equals(prox['pc'],Int(1)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t02 = And(Equals(curr['pc'],Int(0)), Equals(curr['r_'],Int(0)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    t10 = And(Equals(curr['pc'],Int(1)), Equals(prox['pc'],Int(0)),
              Equals(curr['q'], Div(curr['r'],curr['r_'])),
              Equals(prox['r'],curr['r_']), Equals(prox['r_'], Minus(curr['r'],Times(curr['q'],curr['r_']))),
              Equals(prox['s'],curr['s_']), Equals(prox['s_'], Minus(curr['s'],Times(curr['q'],curr['s_']))),
              Equals(prox['t'],curr['t_']), Equals(prox['t_'], Minus(curr['t'],Times(curr['q'],curr['t_']))))

    t22 = And(Equals(curr['pc'],Int(2)), Equals(prox['pc'],Int(2)),
              Equals(prox['r'],curr['r']), Equals(prox['r_'],curr['r_']),
              Equals(prox['s'],curr['s']), Equals(prox['s_'],curr['s_']),
              Equals(prox['t'],curr['t']), Equals(prox['t_'],curr['t_']),
              Equals(prox['q'],curr['q']))

    return Or(t01,t02,t10,t22)

In [13]:
def error1(s):
  return Equals(s["r"],Int(0))

In [14]:
def genTrace(vars,init,trans,error,n):
    with Solver(name="z3") as s:
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]

        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

In [15]:
genTrace(['pc','r','r_','s','s_','t','t_','q'],init1,trans1,error1, 5)

Estado: 0
           pc = 0
           r = 20
           r_ = 15
           s = 1
           s_ = 0
           t = 0
           t_ = 1
           q = 1
Estado: 1
           pc = 1
           r = 20
           r_ = 15
           s = 1
           s_ = 0
           t = 0
           t_ = 1
           q = 1
Estado: 2
           pc = 0
           r = 15
           r_ = 5
           s = 0
           s_ = 1
           t = 1
           t_ = -1
           q = 3
Estado: 3
           pc = 1
           r = 15
           r_ = 5
           s = 0
           s_ = 1
           t = 1
           t_ = -1
           q = 3
Estado: 4
           pc = 0
           r = 5
           r_ = 0
           s = 1
           s_ = -3
           t = -1
           t_ = 4
           q = 0


In [37]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name="z3") as solver:

        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        transt = invert(trans)

        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1])

        # Step 1 implícito na ordem de 'order' e nas definições de Rn, Um.
        for (n,m) in order:
            # Step 2.
            I = init(X[0])
            Tn = And([trans(X[i], X[i+1]) for i in range(n)])
            Rn = And(I, Tn)

            E = error(Y[0])
            Bm = And([transt(Y[i], Y[i+1]) for i in range(m)])
            Um = And(E, Bm)

            Vnm = And(Rn, same(X[n], Y[m]), Um)
            if solver.solve([Vnm]):
                print("> O sistema é inseguro.")
                return
            else:
                # Step 3.
                A = And(Rn, same(X[n], Y[m]))
                B = Um
                C = binary_interpolant(A, B)

                # Salvaguardar cálculo bem-sucedido do interpolante.
                if C is None:
                    print("> O interpolante é None.")
                    break

                # Step 4.
                C0 = rename(C, X[0])
                T = trans(X[0], X[1])
                C1 = rename(C, X[1])

                if not solver.solve([C0, T, Not(C1)]):
                    # C é invariante de T.
                    print("> O sistema é seguro.")
                    return
                else:
                    # Step 5.1.
                    S = rename(C, X[n])
                    while True:
                        # Step 5.2.
                        T = trans(X[n], Y[m])
                        A = And(S, T)
                        if solver.solve([A, Um]):
                            print("> Não foi encontrado majorante.")
                            break
                        else:
                            # Step 5.3.
                            C = binary_interpolant(A, Um)
                            Cn = rename(C, X[n])
                            if not solver.solve([Cn, Not(S)]):
                                # Step 5.4.
                                # C(Xn) -> S é tautologia.
                                print("> O sistema é seguro.")
                                return
                            else:
                                # Step 5.5.
                                # C(Xn) -> S não é tautologia.
                                S = Or(S, Cn)

    print("> Não foi provada a segurança ou insegurança do sistema.")

model_checking(['pc','r','r_','s','s_','t','t_','q'], init1, trans1, error1, 40, 40)

NoSolverAvailableError: ignored